# HuggingFace: Tabular (Question Answering)

In this notebook I will be using the same data used in the Fast AI tabular section where I created a model that implements regression. This section will improve on the work done in that section and apply another preprocessing step in the ETL section. Based on the outcome of the regressor model, splitting the data features for each safeguard subject may help increase the signal of the environmental cost based on revenue. 

This may help in building a new model in this section that is able to answer questions about the data. In the previous section, I read the research that was used to create this data and it seems that each of the individual safeguard subjects contain multiple environmental affect features within them. So splitting these features may help the model understand the underlying distribution for each subject feature and how it affects the environmental cost.

# Imports

In [1]:
import pandas as pd
import numpy as np
import re

## The Data

The dataset contains these main safeguard subjects containing environmental cost according to these industry segments. Previously, the data for each feature was combined into a single numerical value which gave poor results. A better method may be to split the tuple value for each feature into another feature. For example human_health1, human_health2, etc, based on the number of tuple value per record for that feature. Since this is a variable value due to missingness, a better method may be to create the n number of features based on the median length of the tuples for each individual feature and then fill in the records having less than the median with an imputation of the mean for the missing tuple index based on the mean of that tuple index for that feature.

1. Human Health (Working Capacity): Working Capacity captures human health effects from climate change, air pollution, and toxicity.
2. Crop Production : captures soil degradation, air pollution, climate change, and land use.
3. Meat Production Capacity: includes soil degradation, climate change, land use, and toxicity.
4. Fish Production Capacity: includes acidification, eutrophication, climate change, and toxicity.
5. Wood Production Capacity: includes climate change, air pollution, and land use.
6. Drinking Water & Irrigation Water (Water Production Capacity): captures climate change, land use, and water pollution.
7. Abiotic Resources: includes environmental impacts from mining.
8. Biodiversity: captures land use and toxicity.

This information was taken from the dataset source linked in the main readme of this repository.

In [2]:
corp_env_impact_3pct = pd.read_csv('../../data/tabular_data/final_raw_sample_3_percent.csv')

## Extraction, Transformation, Loading

 It is mentioned in the kaggle source that "Each safeguard subject is made up of multiple impact categories and indicators, called state indicators, for measuring the current state of each safeguard subject (Life Cycle Initiative 2016; Steen and Palander 2016)."

Guidance given in the Kaggle source:

'y' target:

- Total Environmental Intensity (Revenue): The monetized environmental impact of the firm's operations during the specific year indicated in column A divided by revenue in that year.

'X' features: 

- Column H: Working Capacity captures human health effects from climate change, air pollution, and toxicity.
- Column I: Fish Production Capacity includes acidification, eutrophication, climate change, and toxicity.
- Column J: Crop Production Capacity captures soil degradation, air pollution, climate change, and land use.
- Column K: Meat Production Capacity includes soil degradation, climate change, land use, and toxicity.
- Column L: Biodiversity captures land use and toxicity.
- Column M: Abiotic Resources includes environmental impacts from mining.
- Column N: Water production capacity captures climate change, land use, and water pollution.
- Column O: Wood Production Capacity includes climate change, air pollution, and land use.

In [3]:
corp_env_impact_3pct.sample(3)

,Year,Company Name,Country,Industry (Exiobase),Total Environmental Intensity (Revenue),Total Environmental Intensity (Operating Income),Total Environmental Cost,Working Capacity,Fish Production Capacity,Crop Production Capacity,Meat Production Capacity,Biodiversity,Abiotic Resources,Water production capacity (Drinking water & Irrigation Water),Wood Production Capacity,% Imputed
567,2018,KISCO CORPORATION,South Korea,Manufacture of basic iron and steel and of fer...,-12.87%,-393.84%,"(8,99,06,545)","(7,68,05,412)","(17,412)","(7,07,477)","(1,60,871)","(2,947)","(87,63,233)","(34,60,430)","11,238",38%
6499,2015,VISCOFAN SA,Spain,Processing of Food products nec,-8.33%,-40.14%,"(6,69,86,147)","(6,05,42,750)","(14,010)","(8,04,400)","(1,88,167)","(2,547)","(1,826)","(54,33,357)",909,7%
12627,2010,CVS HEALTH CORP,United States,"Manufacture of medical,precision and optical i...",-0.29%,-4.47%,"(27,57,09,523)","(19,35,73,214)","(58,862)","(24,05,511)","(4,98,666)","(11,300)","(2,54,698)","(7,90,29,896)","1,22,625",2%


In [4]:
env_features = corp_env_impact_3pct.iloc[:, 7: 14]
# convert the features from str tuple into numerical tuple
for col in env_features.columns:
    env_features[col] = env_features[col].apply(lambda x: tuple(map(int, re.findall(r'\d+', x))))
env_features.columns = [x.lower().replace(' ', '_') for x in env_features.columns]
env_features.head()

,working_capacity,fish_production_capacity,crop_production_capacity,meat_production_capacity,biodiversity,abiotic_resources,water_production_capacity_(drinking_water_&_irrigation_water)
0,"(69, 39, 394)","(1, 763)","(90, 516)","(21, 94)","(310,)","(3, 850)","(2, 40, 3, 212)"
1,"(1, 75, 76, 865)","(4, 191)","(2, 34, 20)","(54, 603)","(766,)","(3, 661)","(56, 57, 645)"
2,"(1, 83, 85, 456)","(4, 273)","(2, 43, 627)","(56, 860)","(783,)","(3, 489)","(33, 90, 60)"
3,"(1, 93, 12, 582)","(4, 522)","(2, 56, 694)","(60, 89)","(820,)","(3, 428)","(1, 5, 662)"
4,"(91, 62, 390)","(3, 149)","(1, 26, 821)","(28, 623)","(597,)","(1, 94, 781)","(47, 45, 756)"


In [5]:
def split_env_feature(feature):
    col_name = feature.name
    calculate_median_tuple_len = feature.apply(len)
    # to calculate the average per idx
    med_ind_map = {f'{col_name}_{x}':0 for x in range(int(calculate_median_tuple_len.median()))}
    num_vals_per_idx = med_ind_map.copy()
    # to hold the new features
    features = {col:[] for col in med_ind_map.keys()}
    def calculate_mean_by_tuple_ind(row,sum_map = med_ind_map,total_n_map=num_vals_per_idx, num_idx = len(features.keys()), features=features):
        for i in range(num_idx):
            if i < len(row):
                sum_map[list(sum_map.keys())[i]] += row[i]
                total_n_map[list(sum_map.keys())[i]] += 1
                features[list(features.keys())[i]].append(row[i])
            else:
                features[list(features.keys())[i]].append(np.nan)
    feature.apply(calculate_mean_by_tuple_ind)
    features = pd.DataFrame(features)
    med_ind_map = {idx_sum[0]:idx_sum[1]/num_vals_per_idx[idx_sum[0]] for idx_sum in med_ind_map.items()}

    # filling in the mean for each split feature using the med_ind_map
    for feature_name in features.columns:
        features[feature_name].fillna(med_ind_map[feature_name], inplace=True)

    return features

In [6]:
corp_safeguard_split = None
for col in env_features.columns:
    if isinstance(corp_safeguard_split, type(None)):
        corp_safeguard_split = split_env_feature(env_features[col])
    else:
        corp_safeguard_split = pd.concat([corp_safeguard_split,split_env_feature(env_features[col])], axis=1)

In [7]:
corp_safeguard_split['Year'] = corp_env_impact_3pct.Year
corp_safeguard_split['Company Name'] = corp_env_impact_3pct['Company Name']
corp_safeguard_split['target'] = corp_env_impact_3pct['Total Environmental Intensity (Operating Income)']

In [8]:
# used to convert the target column to a float percent
def text_percent_to_numeric(text_percent):
    if isinstance(text_percent, type(np.nan)):
        return np.nan
    # Remove percentage sign and convert to float
    numeric_value = float(text_percent.strip('%'))
    # Convert to decimal representation
    numeric_value /= 100
    return numeric_value

In [9]:
corp_safeguard_split['target'] = corp_safeguard_split['target'].apply(text_percent_to_numeric)

In [10]:
corp_safeguard_split.head()

,working_capacity_0,working_capacity_1,working_capacity_2,working_capacity_3,fish_production_capacity_0,fish_production_capacity_1,crop_production_capacity_0,crop_production_capacity_1,crop_production_capacity_2,meat_production_capacity_0,...,biodiversity_0,biodiversity_1,abiotic_resources_0,abiotic_resources_1,water_production_capacity_(drinking_water_&_irrigation_water)_0,water_production_capacity_(drinking_water_&_irrigation_water)_1,water_production_capacity_(drinking_water_&_irrigation_water)_2,Year,Company Name,target
0,69,39,394.0,445.799955,1,763.0,90,516.0,437.036571,21,...,310,418.64977,3.0,850.0,2,40.0,3.0,2018,APPLUS SERVICES SA,-0.2512
1,1,75,76.0,865.000000,4,191.0,2,34.0,20.000000,54,...,766,418.64977,3.0,661.0,56,57.0,645.0,2018,TELEPERFORMANCE SE,-0.0428
2,1,83,85.0,456.000000,4,273.0,2,43.0,627.000000,56,...,783,418.64977,3.0,489.0,33,90.0,60.0,2018,SGS SA,-0.0213
3,1,93,12.0,582.000000,4,522.0,2,56.0,694.000000,60,...,820,418.64977,3.0,428.0,1,5.0,662.0,2018,INTERTEK GROUP PLC,-0.0339
4,91,62,390.0,445.799955,3,149.0,1,26.0,821.000000,28,...,597,418.64977,1.0,94.0,47,45.0,756.0,2018,THOMSON REUTERS CORP,-0.0191


## Model Training & Inference
I will be using the pre-trained BERT Tapas model specifically trained for question answering on tabular data. This model is imported using the transformers library and is sourced from [HuggingFace](https://huggingface.co/docs/transformers/v4.38.2/en/model_doc/tapas#transformers.TapasForQuestionAnswering). The model will then be fine-tuned using the data I prepared and text queries will be prompted for evaluation in the same step.

In [11]:
from transformers import TapexTokenizer, BartForConditionalGeneration, pipeline

/Users/mocha/miniconda3/envs/mamba/envs/fastai_env/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [12]:
corp_safeguard_split.Year.value_counts(normalize=True) # we will create a training condition using data before 2018

2017    0.135539
2018    0.134856
2016    0.131138
2015    0.122334
2014    0.113000
2013    0.105411
2012    0.096456
2011    0.084921
2010    0.076345
Name: Year, dtype: float64

In [13]:
corp_safeguard_split.columns

Index(['working_capacity_0', 'working_capacity_1', 'working_capacity_2',
       'working_capacity_3', 'fish_production_capacity_0',
       'fish_production_capacity_1', 'crop_production_capacity_0',
       'crop_production_capacity_1', 'crop_production_capacity_2',
       'meat_production_capacity_0', 'meat_production_capacity_1',
       'biodiversity_0', 'biodiversity_1', 'abiotic_resources_0',
       'abiotic_resources_1',
       'water_production_capacity_(drinking_water_&_irrigation_water)_0',
       'water_production_capacity_(drinking_water_&_irrigation_water)_1',
       'water_production_capacity_(drinking_water_&_irrigation_water)_2',
       'Year', 'Company Name', 'target'],
      dtype='object')

In [489]:
corp_safeguard_split.to_csv('../../data/tabular_data/corp_safeguard_split.csv', index=False)

In [377]:
corp_safeguard_split_train = corp_safeguard_split.copy().where(corp_safeguard_split.Year < 2018).dropna(how='all')
corp_safeguard_split_test = corp_safeguard_split.copy().where(corp_safeguard_split.Year >= 2018).dropna(how='all')

In [378]:
print(corp_safeguard_split_train.shape)
print(corp_safeguard_split_test.shape)

(11400, 21)
(1777, 21)


### Converting the numerical columns into text
From looking at the model documentation, it seems text large language models prefer numbers to be formatted as strings. I will do that right before input to the model.

In [347]:
tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")

In [348]:
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-large-finetuned-wtq")

In [381]:
corp_safeguard_split_test = corp_safeguard_split_test.astype(str)

In [458]:
question_sample = corp_safeguard_split_test.loc[:50,['Company Name', 'biodiversity_0']]

In [459]:
question_sample.head()

,Company Name,biodiversity_0
0,APPLUS SERVICES SA,310.0
1,TELEPERFORMANCE SE,766.0
2,SGS SA,783.0
3,INTERTEK GROUP PLC,820.0
4,THOMSON REUTERS CORP,597.0


In [460]:
question_sample.shape

(51, 2)

In [476]:
queries = ["Which hong kong company has a medium biodiversity value?"]

In [481]:
for question in queries:
    print('Question: ', question)
    encoding = tokenizer(question_sample, queries=question,return_tensors="pt")
    outputs = model.generate(**encoding)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(result)
    print('Answer: ', result[0].strip())

Question:  Which hong kong company has a medium biodiversity value?
[' nisshinbo holdings incorporated']
Answer:  nisshinbo holdings incorporated


In [482]:
search_pattern = r'^N.*HOLDINGS.*'
result = question_sample[question_sample['Company Name'].str.contains(search_pattern, regex=True)]

In [483]:
result

,Company Name,biodiversity_0
50,NISSHINBO HOLDINGS INCORPORATED,3.0


In [484]:
question_sample.sort_values(by='biodiversity_0', ascending=False)[:10]

,Company Name,biodiversity_0
18,HONG KONG EXCHANGES AND CLEARING LIMITED,92.0
20,BALOISE HOLDING AG,87.0
19,RICARDO PLC,83.0
3,INTERTEK GROUP PLC,820.0
42,FAR EASTERN NEW CENTURY CORPORATION,8.0
2,SGS SA,783.0
1,TELEPERFORMANCE SE,766.0
21,ASX LIMITED,70.0
45,SIEMENS AG,7.0
23,HELVETIA HOLDING AG,63.0
